##26.1 Simple LSTM for Sequence Classification

####Step 1: Import classes and functions and seed random number generator

In [1]:
import numpy
from keras.datasets import imdb
from keras import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

#fix random seed fro reproducibility
numpy.random.seed(7)


####Step 2: Load and split dataset
Split into train(50%) and test(50%) sets

In [2]:
#only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = top_words)

17473536/17464789 [==============================] - 0s 0us/step


####Step 3: Left-pad sequences to all be the same length
truncate and pad the input sequences so that they are all the same length
for modeling

In [3]:
# truncate and pad input sequence
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen= max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

####Step 4: Define and fit lstm model for the IMDB dataset
The first layer is the Embedded layer
that uses 32 length vectors to represent each word. The next layer is the LSTM layer with 100
memory units (smart neurons).

Classification problem = Dense output layer with 1 neuron and sigmoid activation function to make 0/1 predictions for two classes (good/bad)

Binary classification problem = loss function =binary_crossentropy, ADDAM optimization algorithm

3 epochs only = quickly overfits the problem

batch size = 64 to space out weight updates

In [4]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation= 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics= ['accuracy'])
model.summary()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
391/391 [==============================] - 21s 36ms/step - loss: 0.7122 - accuracy: 0.5965 - val_loss: 0.6427 - val_accuracy: 0.6292
Epoch 2/3
391/391 [==============================] - 14s 35ms/step - loss: 0.4795 - accuracy: 0.7704 - val_loss: 0.5763 - val_accuracy: 0.7342
Epoch 3/3
391/391 [=========

####Step 5: Evaluate the Fit model

In [5]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.35%


####FULL CODE

In [6]:
# LSTM for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation= 'sigmoid' ))
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
model.summary()
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 32)           160000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               53200     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
391/391 [==============================] - 12s 26ms/step - loss: 0.5051 - accuracy: 0.7495
Epoch 2/3
391/391 [==============================] - 10s 25ms/step - loss: 0.3367 - accuracy: 0.8639
Epoch 3/3
391/391 [==============================] - 10s 26ms/step - loss: 0.2700 - accuracy: 0.8933
Accuracy:

##26.2 LSTM for Sequence Classification with Dropout
Recurrent Neural networks like LSTM generally have the problem of overfitting. Dropout can
be applied between layers using the Dropout Keras layer.

Add new Dropout layers between the Embedding and LSTM layers and the LSTM and Dense output layers

In [7]:
# LSTM with Dropout for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32

model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
#***************************add Dropout
model.add(Dropout(0.2))
model.add(LSTM(100))
#***************************add Dropout
model.add(Dropout(0.2))
model.add(Dense(1, activation= 'sigmoid' ))

model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
model.summary()
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 32)           160000    
                                                                 
 dropout (Dropout)           (None, 500, 32)           0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               53200     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
391/391 [=============================

Droput having desired impact on training with slightly slower trend in convergence and lower final accuracy. 
To achieve higher skill 
1. more epochs
2. dropout applied to the input and recurrent connections of the memory units with the LTSM precisely and separately

Keras provides this capability with parameters on the LSTM layer,
the dropout for configuring the input dropout and recurrent dropout for configuring the
recurrent dropout.

In [8]:
# LSTM with dropout for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
# *****************more precise LSTM droupout
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation= 'sigmoid' ))
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
model.summary()
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 32)           160000    
                                                                 
 lstm_3 (LSTM)               (None, 100)               53200     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
391/391 [==============================] - 749s 2s/step - loss: 0.5005 - accuracy: 0.7440
Epoch 2/3
391/391 [==============================] - 740s 2s/step - loss: 0.3433 - accuracy: 0.8583
Epoch 3/3
391/391 [==============================] - 736s 2s/step - loss: 0.3063 - accuracy: 0.8710
Accuracy: 85

##26.3 LSTM and CNN for Sequence Classification
We can easily add a one-dimensional CNN and max pooling layers after the Embedding layer which then feed the consolidated features to the LSTM. We can use a smallish set of 32 features with a small
filter length of 3. The pooling layer can use the standard length of 2 to halve the feature map
size.

In [10]:
# LSTM and CNN for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
# ************add 1-d CNN and max pooling layers
model.add(Conv1D(filters=32, kernel_size=3, padding= 'same' , activation= 'relu' ))
model.add(MaxPooling1D(pool_size=2))

model.add(LSTM(100))
model.add(Dense(1, activation= 'sigmoid' ))
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
model.summary()
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 500, 32)           160000    
                                                                 
 conv1d_1 (Conv1D)           (None, 500, 32)           3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 250, 32)          0         
 1D)                                                             
                                                                 
 lstm_5 (LSTM)               (None, 100)               53200     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
________________________________________________